6. Edge detection - Sobel filter

In [ ]:
# 6 a) Calculate the first derivatives of the image in the x and y direction, using the Sobel function

# Load the image

"""
" @param kernelSize The size of the kernel to use for the Sobel filter
"""
def sobelFilter(imageSrc, kernelSize = 3, useGaussianBlur = False, gaussianBlurKernelSize = 3):
    img = cv2.imread(imageSrc)
    # Check if image was loaded correctly
    if img is None:
        print("error opening image" + imageSrc)
    else:
        # Can apply a gaussian blur to reduce noise
        if useGaussianBlur:
            img = cv2.GaussianBlur(img, (gaussianBlurKernelSize, gaussianBlurKernelSize), 0)

        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Define variables for the Sobel function
        ddepth = cv2.CV_16S
        scale = 1
        delta = 0

        # Calculate the first derivate in the x direction
        grad_x = cv2.Sobel(gray_img, ddepth, 1, 0, ksize= kernelSize, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

        # Calculate the first derivate in the y direction
        grad_y = cv2.Sobel(gray_img, ddepth, 0, 1, ksize= kernelSize, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
        # Could also try the cv2.Scharr function to approximate the derivatives

        # print("grad.x.shape = " + str(grad_x.shape))
        # print("grad.y.shape = " + str(grad_y.shape))
        print("grad_x" + str(grad_x))
        print("grad_y" + str(grad_y))

        return grad_x, grad_y


imageSrc = os.path.join(dataDir, 'images/FEUP_01.jpg')

# Call the method and get the gradients for each axis
grad_x, grad_y = sobelFilter(imageSrc, 3)

grad_x[[  0   2  -2 ...  50  56   0]
 [  0   2   0 ...  55  66   0]
 [  0   1   2 ...  59  73   0]
 ...
 [  0 -25  -6 ...   1   1   0]
 [  0 -10 -11 ...  -1   0   0]
 [  0   0 -14 ...  -4   0   0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 4  4  6 ...  3 18 28]
 [ 2  1  0 ... 13  9  6]
 ...
 [22 27 30 ... -1 -1 -2]
 [18 28 35 ...  1 -2 -2]
 [ 0  0  0 ...  0  0  0]]


In [ ]:
# 6b) Calculate the approximate value of the gradient by combining the directional directives

def combineSobelGradients(grad_x, grad_y, weight1 = 0.5, weight2 = 0.5):
    # Calculate the gradient magnitude in the x and y direction
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)

    # Combine the directional derivatives
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

    # print the combined gradient
    print("grad.shape" + str(grad.shape))
    print("grad" + str(grad))
    return grad

# Call the method to combine the gradients
combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)

grad.shape(787, 1181)
grad[[ 0  1  1 ... 25 28  0]
 [ 2  3  3 ... 29 42 14]
 [ 1  1  1 ... 36 41  3]
 ...
 [11 26 18 ...  1  1  1]
 [ 9 19 23 ...  1  1  1]
 [ 0  0  7 ...  2  0  0]]


In [ ]:
# 6c) Show the gradient image

cv2.imshow("Sobel derivatives image", combinedGrad)

# close the window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Show the result of thresholding the gradient image; use a trackbar to select the threshold value

MAX_VALUE = 255

def showSobelWithThreshold(grad, window_name = "Thresholded gradient image"):
    # Define a function to callback when the trackbar is changed
    def on_trackbar(new_val):
        # Threshold the gradient image
        _ret, thresholdedGradImg = cv2.threshold(grad, new_val, MAX_VALUE, cv2.THRESH_BINARY)
        cv2.imshow(window_name, thresholdedGradImg)

    # Define the window
    cv2.namedWindow(window_name)

    # Create the trackbar
    trackbar_name = "Threshold Trackbar"
    cv2.createTrackbar(trackbar_name, window_name, 0, MAX_VALUE, on_trackbar)

    # Show the initial image
    cv2.imshow(window_name, grad)
        
    # Close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Call the method to show the thresholded gradient image
showSobelWithThreshold(combinedGrad)

In [ ]:
# 6e) try different kernel sizes
for i in range(1, 8, 2):
    grad_x, grad_y = sobelFilter(imageSrc, i)
    combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)

    # Show the gradient image with threshold trackbar and current kernel size
    windowName = "Sobel derivatives image with kernel size " + str(i)
    showSobelWithThreshold(combinedGrad, windowName)

    # close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

grad_x[[ 0  0 -1 ... 10 10  0]
 [ 0  1  0 ... 15 18  0]
 [ 0  0  1 ... 15 20  0]
 ...
 [ 0 -8 -1 ...  0  0  0]
 [ 0 -2 -3 ...  1  0  0]
 [ 0  2 -4 ... -3  0  0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  0  4 10]
 [ 1  0  0 ...  3  3  0]
 ...
 [ 4  7  9 ...  0  0 -1]
 [ 1  8 11 ... -1  0 -1]
 [ 0  0  0 ...  0  0  0]]
grad.shape(787, 1181)
grad[[ 0  0  0 ...  5  5  0]
 [ 0  1  0 ...  8 11  5]
 [ 0  0  0 ...  9 12  0]
 ...
 [ 2  8  5 ...  0  0  0]
 [ 0  5  7 ...  1  0  0]
 [ 0  1  2 ...  2  0  0]]
grad_x[[  0   2  -2 ...  50  56   0]
 [  0   2   0 ...  55  66   0]
 [  0   1   2 ...  59  73   0]
 ...
 [  0 -25  -6 ...   1   1   0]
 [  0 -10 -11 ...  -1   0   0]
 [  0   0 -14 ...  -4   0   0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 4  4  6 ...  3 18 28]
 [ 2  1  0 ... 13  9  6]
 ...
 [22 27 30 ... -1 -1 -2]
 [18 28 35 ...  1 -2 -2]
 [ 0  0  0 ...  0  0  0]]
grad.shape(787, 1181)
grad[[ 0  1  1 ... 25 28  0]
 [ 2  3  3 ... 29 42 14]
 [ 1  1  1 ... 36 41  3]
 ...
 [11 26 18 ...  1  1  1]
 [ 

In [ ]:
# 6f) Test the effect of applying a Gaussian blur before applying the Sobel filter, use gaussian filters with increasing sizes 
# (ex. 3x3, 7x7, 11x11, 31x31)
for kernelSize in range(1, 8, 2):
    for gaussFilterSize in [3, 7, 11, 31]:
        grad_x, grad_y = sobelFilter(imageSrc, kernelSize, True, gaussFilterSize)
        combinedGrad = combineSobelGradients(grad_x, grad_y, 0.5, 0.5)

        # Show the gradient image with threshold trackbar and current kernel size
        windowName = f"Sobel derivatives image with kernel size {str(kernelSize)} and gaussFilterSize {str(gaussFilterSize)}" 
        showSobelWithThreshold(combinedGrad, windowName)

        # close the window
        cv2.waitKey(0)
        cv2.destroyAllWindows()

grad_x[[ 0  0  0 ... 10 10  0]
 [ 0  0 -1 ... 12 11  0]
 [ 0  0  0 ... 13 12  0]
 ...
 [ 0 -3 -3 ... -2  0  0]
 [ 0 -3 -3 ... -3  0  0]
 [ 0 -1 -3 ... -4  0  0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  2  3  4]
 [ 0  0  0 ...  1  1  1]
 ...
 [ 6  6  6 ... -1 -1 -2]
 [ 3  5  5 ...  0 -1  0]
 [ 0  0  0 ...  0  0  0]]
grad.shape(787, 1181)
grad[[0 0 0 ... 5 5 0]
 [0 0 1 ... 7 7 2]
 [0 0 0 ... 7 6 0]
 ...
 [3 4 4 ... 2 0 1]
 [2 4 4 ... 2 0 0]
 [0 0 2 ... 2 0 0]]
grad_x[[ 0 -1 -1 ...  6  4  0]
 [ 0  0 -1 ...  6  5  0]
 [ 0  0  0 ...  5  5  0]
 ...
 [ 0  0 -1 ... -3 -1  0]
 [ 0  0 -1 ... -3 -1  0]
 [ 0 -1 -1 ... -3 -2  0]]
grad_y[[ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  1]
 [ 0  0  0 ...  1  0  0]
 ...
 [ 5  5  5 ... -1 -2 -2]
 [ 3  3  2 ...  0  0 -1]
 [ 0  0  0 ...  0  0  0]]
grad.shape(787, 1181)
grad[[0 0 0 ... 3 2 0]
 [0 0 1 ... 3 2 0]
 [0 0 0 ... 3 2 0]
 ...
 [2 2 3 ... 2 2 1]
 [2 2 2 ... 2 0 0]
 [0 0 0 ... 2 1 0]]
grad_x[[ 0 -1 -1 ...  2  2  0]
 [ 0  0 -1 ...  3  2  0]
 [ 0

7. Edge Detection - Canny filter

In [ ]:
# 7a)
